In [8]:
import pandas as pd
import numpy as np
import datetime

import re

In [105]:
df = pd.read_csv('tripadvisor.csv')

In [106]:
df.head()

,Title,Review,Date,Rating
0,All ok except the cost of parking,We had a budget 2 night stay and its a well se...,Date of stay: April 2016,"<span class=""ui_bubble_rating bubble_40""></span>"
1,Bad experience overall,I had the worse experience ever. The furniture...,Date of stay: April 2016,"<span class=""ui_bubble_rating bubble_20""></span>"
2,Better than expected,"stayed as part of a national coach trip ,pleas...",Date of stay: April 2016,"<span class=""ui_bubble_rating bubble_40""></span>"
3,Dismal,I booked a double room six months in advance o...,Date of stay: April 2016,"<span class=""ui_bubble_rating bubble_10""></span>"
4,Good value for weekend break,"Service was very good, room was large and comf...",Date of stay: April 2016,"<span class=""ui_bubble_rating bubble_40""></span>"


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 4 columns):
Title     1656 non-null object
Review    1656 non-null object
Date      1650 non-null object
Rating    1656 non-null object
dtypes: object(4)
memory usage: 51.9+ KB


In [109]:
#remove nan rows (6 rows with no date)
df = df.dropna()

df.isna().sum()

Title     0
Review    0
Date      0
Rating    0
dtype: int64

In [147]:
df_test = df.copy()

#combine Title and Review
df_test['Review'] = df_test['Title'] + ' ' + df_test['Review']
df_test['Review'] = df_test['Review'].astype(str)
df_test = df_test.drop(columns='Title')

#extract date
def date_process(text):
    return re.sub(r"Date of stay:", "", text)

def remove_whitespace(text):
    return " ".join(text.split())

df_test['Date'] = df_test['Date'].astype(str)
df_test['Date'] = df_test['Date'].apply(date_process)
df_test['Date'] = df_test['Date'].apply(remove_whitespace)

#extract rating
def rating_process(text):
    numeric_filter = filter(str.isdigit, text)
    numeric_string = "".join(numeric_filter)
    return numeric_string

df_test['Rating'] = df_test['Rating'].apply(rating_process)
df_test['Rating'] = df_test['Rating'].astype(int)

In [159]:
#classify into positive=1 for rating >= 40, 0 otherwise
df_test['Positive'] = np.where(df_test['Rating'] >= 40, 1, 0)

In [161]:
df_test.head()

,Review,Date,Rating,Positive
0,All ok except the cost of parking We had a bud...,April 2016,40,1
1,Bad experience overall I had the worse experie...,April 2016,20,0
2,Better than expected stayed as part of a natio...,April 2016,40,1
3,Dismal I booked a double room six months in ad...,April 2016,10,0
4,Good value for weekend break Service was very ...,April 2016,40,1


In [163]:
df_test.to_csv('tripadvisor_processed.csv', header=True, index=False)